In [160]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb
%run ./4-ImplementationBP.ipynb

./img/3/graph.html
./img/3/graph.html


In [161]:
def delta_funct(S_ki, t_i):
    if (len(S_ki) == 0):
        return 0
    if (t_i == np.min(S_ki)):
        return 1
    return 0

In [162]:
def R_i(t_i, r_i, lambda_i):
    if (t_i > r_i):
        return 0
    result = lambda_i*np.exp(-lambda_i*(r_i - t_i))
    return result

In [163]:
def S_ij(s_ij, t_i, r_i, lambda_i, time_range, contact_range_ij):
    if (t_i <= s_ij < r_i):
            temp = np.where((contact_range_ij < s_ij) & (contact_range_ij >= t_i))
            return (lambda_i)*(1-lambda_i)**(len(temp))
    elif (s_ij ==9):
        index_larger_than_t_i = np.where(contact_range_ij >= t_i)
        return (1-lambda_i)**(len(index_larger_than_t_i))
    else:
        return 0

In [164]:
#debug function
def Find_Zero(delt, R, S):
    if (delt == 0):
        print("delt = 0")
    if (R == 0):
        print("R = 0")
    if (S == 0):
        print("S = 0")

def phi(t_i,r_i, s, type):
    delt = delta_funct([s[1-type]],t_i)
    R = R_i(t_i, r_i, 0.3)
    S = S_ij(s[type], t_i, r_i, 0.3, "nothing", np.array([0,6,9]))
    #Find_Zero(delt, R, S)
    return delt*R*S


In [165]:
#phi(t_i,r_i,(s_ij,s_ji), type)

In [166]:
phi(1, 7, (6,1), 0)

0.010413829957959952

In [167]:
#inference using BP

In [168]:
variables_i = ["t_i","r_i","s_ij:s_ji"]
variables_j = ["t_j", "r_j","s_ij:s_ji"]

In [169]:
shape = (10,10,10,10)
matrix_phi_i = np.zeros(shape=(10,10,10))
matrix_phi_j = np.zeros(shape=(10,10,10))
for index in np.ndindex(shape):
    matrix_phi_i[index[0], index[1], index[2]] = phi(index[0], index[1], (index[2], index[3]),0)
    matrix_phi_j[index[0], index[1], index[3]] = phi(index[0], index[1], (index[2], index[3]),1)

In [170]:
phi_i = factor(variables_i, matrix_phi_i)
phi_j = factor(variables_j, matrix_phi_j)

In [171]:
graph = factor_graph()
graph.add_factor_node("phi_i", phi_i)
graph.add_factor_node("phi_j", phi_j)

In [172]:
bp = belief_propagation(graph)
bp_result = bp.belief("t_i").get_distribution()

In [173]:
bp_result

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [174]:
plot_factor_graph(graph)

./img/3/graph.html


In [175]:
#exact calculation of marginals

In [176]:
t_i_array = []
sum = 0
shaper = (10,10,10,10,10)
for t_i in range(10):
    sum = 0
    for index in np.ndindex(shaper):
        r_i = index[0]
        t_j = index[1]
        r_j = index[2]
        s_ij = index[3]
        s_ji = index[4]
        s = (s_ij, s_ji)
        sum += phi(t_i, r_i, s, 0)*phi(t_j, r_j, s, 1)
    t_i_array.append(sum)

In [177]:
sum_Z = np.sum(t_i_array)

In [178]:
t_i_array

[0.02821417899592995,
 0.026809746543156463,
 0.024970822880290698,
 0.02259214462917218,
 0.019570052842432073,
 0.01583464979256541,
 0.01141918132252637,
 0.006601023169757597,
 0.0021782333836571906,
 0.04409999999999999]

In [179]:
marginal_t_i = t_i_array/sum_Z
marginal_t_i

array([0.1394739 , 0.13253123, 0.1234407 , 0.11168195, 0.09674255,
       0.07827696, 0.05644955, 0.03263148, 0.01076787, 0.21800382])

In [180]:
bp_result

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])